# DEPENDENCIAS

Puedes ejecutar una vez este codigo y luego borrarlo.

# Librerias

In [8]:
#import requests
import pandas as pd
from bs4 import BeautifulSoup as bs

import random

import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

import undetected_chromedriver as uc

from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel




# Entradas

In [ ]:
# Ruta local al driver de Chrome
ruta_driver = r"chromedriver-win64/chromedriver.exe"

# Configuración de opciones del navegador
chrome_options = Options()
# chrome_options.add_argument('--headless')  # Ejecutar sin abrir ventana
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--no-sandbox')

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Obtención de Enlaces

In [ ]:
# Abre Google
service = Service(executable_path=ruta_driver)
browser = webdriver.Chrome(service=service, options=chrome_options)

# Esta variable solo servira para asignarle nombre al Excel que se generara al final.
# Te recomiendo ponerle algo con lo que lo diferencies de los demás.
busqueda = 'Software Architect'

# AQUI poner el enlace de GLASSDOOR con el trabajo ya buscado:
# Abres ESTE mismo enlace
# Buscas el trabajo que necesitas "Scrappear"
# Copias el enlace principal y lo pegas acá.
url = 'https://www.glassdoor.com.mx/Empleo/estados-unidos-software-architect-empleos-SRCH_IL.0,14_IN1_KO15,33.htm'

print(url)

# Se ingresa el URL asignado
browser.get(url)

# Obtención de estructura HTML
html = browser.page_source
soup = bs(html, 'lxml')

# Filtrado de enlaces
articles = soup.find('ul', {'class': 'JobsList_jobsList__lqjTr'}).find_all('a', {'class': 'JobCard_jobTitle__GLyJ1'})
link_chambas = [article.get('href') for article in articles]

browser.quit()

# Selecciona 15 enlaces aleatorios de la lista
# Este lo puedes ajustar como gustes, yo puse 15 para tener de donde escoger.
link_chambas = random.sample(link_chambas, min(len(link_chambas), 10))
link_chambas



## Scrapping de Trabajos y Exportación EXCEL

In [ ]:
data = []

for url in link_chambas:

    service = Service(executable_path=ruta_driver)
    browser = webdriver.Chrome(service=service, options=chrome_options)
    browser.get(url)

    wait_time = random.uniform(3, 6)
    print(f"Esperando {wait_time:.2f} segundos antes de continuar...")
    time.sleep(wait_time)

    soup = bs(browser.page_source, 'html.parser')

    vacante = {}

    try:
        info = soup.find('div',{'class':'JobDetails_jobDetailsContainer__y9P3L'})
        vacante["Descripción"] = info.get_text(separator='\n').strip()
    except AttributeError:
        print(f"No se encontró la info en: {url}")

    vacante["Enlace"] = url  # Esto no genera error, así que no necesitas try/except aquí

    data.append(vacante)
    print(vacante)

    print("")
    browser.quit()

# Eliminar fragmento de texto especifico que siempre sale.
for vacante in data:
    vacante["Descripción"] = vacante["Descripción"].replace("Postularse en el sitio del empleador\nPostúlate ahora", "")

# Guardar datos como JSON estructurado y legible
nombre_archivo = f"GlassDoor_{busqueda.replace(' ', '_')}.json"
with open(nombre_archivo, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

print("Los datos se han exportado correctamente como JSON.")


# Open AI USE

In [ ]:
class structure(BaseModel):
    place: str
    company: str
    position: str
    academic_profile: str
    english_level: str
    technical_skills: str
    soft_skills: str
    salary: str
    source: str
    link: str

In [ ]:
resultados = []
df = pd.DataFrame()
df_vacantes = pd.DataFrame(data)

columnas_estructuradas = structure.model_fields.keys()
for col in columnas_estructuradas:
    df_vacantes[col] = None

for idx, row in df_vacantes.iterrows():
    texto = row["Descripción"]
    enlace = row["Enlace"]
    print(f"Procesando vacante {idx+1} de {len(df_vacantes)}: {enlace}")


    try:
        response = client.responses.parse(
            model = "gpt-5",
            input = [
                {
                    #Instrucciones para el asistente
                    "role": "system",
                    "content": "Eres un asistente experto en extraer información de ofertas laborales. Tu tarea es analizar el texto proporcionado y generar una estructura con los campos requeridos sin alterar el texto original (A menos que se te especifique, como es el caso de English_level). No debes incluir ningún comentario o explicación adicional, solo el contenido estructurado con la información extraída. En el campo de 'english_level', si no se menciona explicitamente pero se infiere que el puesto requiere un nivel de inglés avanzado (por ejemplo, que el pais de la solicitud sea angloparlante), debes asignar 'Nativo'. Si no se menciona el nivel de inglés, asigna 'No especificado'. El apartado de 'source' siempre sera 'Glassdoor'.",
                },
                {
                    "role": "user",
                    "content": f"Usa el siguiente texto para extraer la información requerida:\n\n{texto}, enlace: {enlace}"
                },
            ],
            temperature=0.2,
            text_format=structure,
        )
        
        resultados = pd.DataFrame(response.output_parsed)
        resultados = resultados.transpose() # Transponer el DataFrame
        resultados.columns = resultados.iloc[0] # Columna

        resultados = resultados[1:]  # Eliminar la primera fila que ahora es el encabezado
        df = pd.concat([df, resultados], ignore_index=True)
        print("Trabajos formateados exitosamente.")
        df
    


    except Exception as e:
        print(f"Error al procesar la vacante {enlace}")
        print(e)


---

In [ ]:
df

In [ ]:
# Exportar datos a Excel
df.to_excel(f"sad{busqueda.replace(' ', '_')}.xlsx", index=False, engine="openpyxl")